In [ ]:
# User-defined variables
work_dir = '/Users/Tem/Documents/naturalistic-threat-ptsd'
hemo_lag = 4
nPerms = 10000
trim_TRs = 4 #number of TRs to trim from beginning and end of naturalistic scan (separate from hemodynamic adjustment)

In [ ]:
# Import packages 
import warnings
warnings.filterwarnings("ignore")
import os
import random
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
from scipy.stats import iqr
from datetime import datetime
import matplotlib.pyplot as plt
from matplotlib import gridspec
from statsmodels.stats.multitest import fdrcorrection

In [ ]:
# Create directory to house comparison figures
date_string = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
comparison_fig_dir = f"{work_dir}/data/figures/comparison-figs-{date_string}"
os.makedirs(comparison_fig_dir)

In [ ]:
# Load movie metadata 
movie_metadata = pd.read_csv(f"{work_dir}/data/task/stim-metadata-naturalistic.csv") 
movie_metadata['Start_TR'][0] = movie_metadata['Start_TR'][0] + trim_TRs
movie_metadata['Stop_TR'][len(movie_metadata) - 1] = movie_metadata['Stop_TR'][len(movie_metadata) - 1] - trim_TRs
movie_metadata['Scene_Length'] = movie_metadata['Stop_TR'] - movie_metadata['Start_TR']

nScenes = len(movie_metadata)
movie_TRs = [list(range(movie_metadata['Start_TR'][i], movie_metadata['Stop_TR'][i])) for i in range(0, nScenes)]

In [ ]:
# Import symptom data
CAPS_data = pd.read_csv(f"{work_dir}/data/subs/CAPS.csv")
cohort_IDs = pd.read_csv(f"{work_dir}/data/subs/cohort_IDs.csv")
nCohort = len(cohort_IDs)
symptom_labels = ['Hyperarousal']

## Left Amygdala Classical InterSC

In [ ]:
# Load event files for classical stimulus onsets
all_CSminus_onsets = [None] * nCohort
all_CSplus_onsets = [None] * nCohort

for i in range(nCohort):
    day1_onsets = pd.read_csv(f"{work_dir}/data/task/events-classical/{cohort_IDs['subject'][i]}.csv")
    
    day1_CSplus = day1_onsets[day1_onsets['trial_type'].str.contains('plus') & ~day1_onsets['trial_type'].str.contains('US')].reset_index()
    CSplus_onsets = [list(range(day1_CSplus['onset'][i] + hemo_lag, day1_CSplus['onset'][i] + day1_CSplus['duration'][i] + hemo_lag)) for i in range(len(day1_CSplus))]
    all_CSplus_onsets[i] = [item for sublist in CSplus_onsets for item in sublist]

    day1_CSminus = day1_onsets[day1_onsets['trial_type'].str.contains('minus')].reset_index()
    CSminus_onsets = [list(range(day1_CSminus['onset'][i] + hemo_lag, day1_CSminus['onset'][i] + day1_CSminus['duration'][i] + hemo_lag)) for i in range(len(day1_CSminus))]
    all_CSminus_onsets[i] = [item for sublist in CSminus_onsets for item in sublist]
    

In [ ]:
# Calculate classical interSC 
classical_data = [None] * nCohort

for i in range(nCohort):
    classical_data[i] = np.load(work_dir +
                                '/data/neural-classical/amygdala-harvard-oxford-6mm/HO_1_6mm/' +
                                cohort_IDs['subject'][i] + '_6mm.npy')

nVoxels = classical_data[0].shape[0]
classical_patterns = np.zeros((nVoxels, 2, nCohort))

for i in range(nCohort):
    classical_patterns[:, 0, i] = np.mean(classical_data[i][:, all_CSminus_onsets[i]], axis = 1)
    classical_patterns[:, 1, i] = np.mean(classical_data[i][:, all_CSplus_onsets[i]], axis = 1)
    
subs = np.arange(0, nCohort)
classical_heldout = np.zeros_like(classical_patterns)
    
for i in subs:
    sel_subs = subs[subs!= i]
    classical_heldout[:, :, i] = np.mean(classical_patterns[:, :, sel_subs], axis = 2)
    
all_mats = np.zeros((2, 2, nCohort))
LA_interSC = np.zeros((2, nCohort))

for i in range(nCohort):
    sub_mat = np.corrcoef(classical_patterns[:,:,i], classical_heldout[:,:,i], rowvar = False)
    sub_mat = sub_mat[:2, 2:]
    all_mats[:,:,i] = sub_mat
    LA_interSC[:, i] = np.diagonal(sub_mat)
    
# Calculate and print median interSC values
threat_threat_isc = np.zeros(nCohort)
for i in range(nCohort):
    threat_threat_isc[i] = all_mats[:,:,i][1,1]
threat_threat_med = np.median(threat_threat_isc)
print(f"median classical interSC: {threat_threat_med}")

## Left Amygdala Naturalistic InterSC

In [ ]:
# Define function that calculates spatial interSC for naturalistic data
def calc_spatial_interSC(group_data):

    nGroup = group_data.shape[2]
    
    group_ByScene = np.zeros((nVoxels, nScenes, nGroup))

    for i in range(0, nScenes):
        scene_TRs = (movie_TRs[i])
        scene_TRs = [x - 1 for x in scene_TRs] 
        group_ByScene[:,i,:] = np.mean(group_data[:, scene_TRs, :], axis = 1)
    
    subs = np.arange(0, nGroup)
    group_heldout = np.zeros((nVoxels, nScenes, nGroup))
    
    for i in subs:
        sel_subs = subs[subs!= i]
        group_heldout[:, :, i] = np.mean(group_ByScene[:, :, sel_subs], axis = 2)
        
    full_interSC_group = np.zeros((nScenes, nGroup))
    group_median_nonmatch = np.zeros(nGroup)

    diag_mask = np.ones((nScenes, nScenes), dtype = bool)
    np.fill_diagonal(diag_mask, False)
    div_by = len(diag_mask)

    for i in range(nGroup):
        sub_mat = np.corrcoef(group_ByScene[:,:,i], group_heldout[:,:,i], rowvar = False)
        sub_mat = sub_mat[div_by:, :div_by]
        full_interSC_group[:, i] = np.diagonal(sub_mat)
        group_median_nonmatch[i] = np.median(sub_mat[diag_mask])
    
    return full_interSC_group, group_ByScene, group_median_nonmatch

In [ ]:
# Load left amygdala naturalistic data
data_dir = f"{work_dir}/data/neural-naturalistic/amygdala-harvard-oxford-6mm"
list_LA_data = [None] * nCohort
for i in range(len(cohort_IDs)):
    list_LA_data[i] = np.load(f"{data_dir}/HO_1_6mm/{cohort_IDs['subject'][i]}_6mm.npy")
LA_data = np.stack(list_LA_data, axis = 2)

# Calculate naturalistic interSC 
nVoxels = LA_data.shape[0]
LA_full_interSC, LA_ByScene, LA_nonmatch = calc_spatial_interSC(LA_data)
LA_interSC = np.median(LA_full_interSC, axis = 0)
print(f"median naturalistic interSC: {np.median(LA_interSC)}")

## Naturalistic vs. Classical Spearman's coefficients

In [ ]:
# Define function that correlates an interSC variable with each dimension of a symptom variable
def corr_isc_symp(interSC_var, symp_var): 
    actual_corrs = [None] * len(symptom_labels)
    corr_p_vals = [None] * len(symptom_labels)
    all_null_dists = [None] * len(symptom_labels)
    
    for i in range(len(symptom_labels)):
        which_score = symptom_labels[i]
        null_dist = np.zeros(nPerms)

        # Actual r value
        x = symp_var[which_score]
        y = interSC_var
        actual_corr = stats.spearmanr(x, y)[0]
        actual_corrs[i] = actual_corr

        # Null dist of r-values 
        for j in range(nPerms):
            x = symp_var[which_score]
            y = interSC_var[random.choices(range(nCohort), k = (nCohort))]
            null_dist[j] = stats.spearmanr(x, y)[0]
    
        perms_above_frac = len((null_dist[null_dist > actual_corr]) + 1) / (nPerms + 1)
        corr_p_vals[i] = min(2 * perms_above_frac, 2 * (1 - perms_above_frac))
        all_null_dists[i] = null_dist
        
    return actual_corrs, corr_p_vals, all_null_dists

In [ ]:
classical_rho, classical_p_val, classical_nulls =  corr_isc_symp(threat_threat_isc, CAPS_data)
naturalistic_rho, naturalistic_p_val, naturalistic_nulls =  corr_isc_symp(LA_interSC, CAPS_data)

actual_diff = naturalistic_rho[0] - classical_rho[0]
null_dist = naturalistic_nulls[0] - classical_nulls[0]

perms_above_frac = len((null_dist[null_dist > actual_diff]) + 1) / (nPerms + 1)
p = min(perms_above_frac, (1 - perms_above_frac))

In [ ]:
f = plt.figure(figsize=(5, 6))
x = [0.25, 0.75]
y = [naturalistic_rho[0], classical_rho[0]]
sns.barplot(x = x, y = y, color = "#8c0e88")
plt.xticks([])
plt.yticks(fontsize = 16)
plt.ylabel(f"Pearson's r \nInterSC vs. Hyperarousal\n", fontsize = 16)
sns.despine(top = False, bottom = True)
plt.subplots_adjust(left = 0.15)
plt.savefig(f"{comparison_fig_dir}/barplots.png")

In [ ]:
f = plt.figure(figsize=(9, 5))
plt.xticks(fontsize = 16)
plt.yticks(fontsize = 16)
sns.histplot(data = null_dist, color = '0.25', element = "step", fill = False, linewidth = 3)
plt.xlabel(f"\np = {np.round(p, 5)}", fontsize = 12)
plt.axvline(actual_diff, color = '#8c0e88', linewidth = 2.5)
plt.axvline(np.mean(null_dist), color = '0.5', linestyle = 'dashed', linewidth = 2)
plt.ylabel(None)
sns.despine()
plt.savefig(f"{comparison_fig_dir}/histogram.png")